In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import check_output

In [26]:
data = pd.read_csv('train.csv',nrows=10000,dtype={'is_booking':bool,'srch_destination_id':np.int32, 'hotel_cluster':np.int32},
                    usecols=['srch_destination_id','is_booking','hotel_cluster'],chunksize=10000)
# data.head(10)
aggs = []
print('-'*38)
for chunk in data:
    agg = chunk.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count'])
    agg.reset_index(inplace=True)
    aggs.append(agg)
    print('.',end='')
print('')
aggs = pd.concat(aggs, axis=0)
aggs.head()

--------------------------------------
.


,srch_destination_id,hotel_cluster,sum,count
0,11,94,1.0,2
1,16,7,1.0,2
2,19,20,0.0,1
3,19,30,0.0,1
4,19,40,0.0,4


In [27]:
CLICK_WEIGHT = 0.05
agg = aggs.groupby(['srch_destination_id','hotel_cluster']).sum().reset_index()
agg['count'] -= agg['sum']
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
agg.head()

,srch_destination_id,hotel_cluster,bookings,clicks,relevance
0,11,94,1.0,1.0,1.05
1,16,7,1.0,1.0,1.05
2,19,20,0.0,1.0,0.05
3,19,30,0.0,1.0,0.05
4,19,40,0.0,4.0,0.20


In [28]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

In [29]:
most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
srch_destination_id,
11,94
16,7
19,40 61 30 20
56,57
66,50


In [30]:
test = pd.read_csv('test.csv',
                    dtype={'srch_destination_id':np.int32},
                    usecols=['srch_destination_id'],)

In [31]:
test = test.merge(most_pop, how='left',left_on='srch_destination_id',right_index=True)
test.head()

,srch_destination_id,hotel_cluster
0,12243,NaN
1,14474,NaN
2,11353,21 91 84 59 48
3,8250,1 45 54 88 24
4,11812,47 16
